In [ ]:
!pip install -U transformers>=4.48.0

In [ ]:
!pip install transformers datasets seqeval --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_

In [ ]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
import torch

In [ ]:
from google.colab import files
import pandas as pd
import ast
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import BertTokenizerFast
import numpy as np
from transformers import BertForTokenClassification, TrainingArguments, Trainer
from seqeval.metrics import f1_score
# import torch

In [ ]:
# Load the CSV files assuming keys "train.csv" and "test.csv" in uploaded dict
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

# Parse the stringified lists (adjust column names if different)
train_df['Sentence'] = train_df['Sentence'].apply(ast.literal_eval)
train_df['NER Tag'] = train_df['NER Tag'].apply(ast.literal_eval)
test_df['Sentence'] = test_df['Sentence'].apply(ast.literal_eval)

print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)

Train shape: (40000, 3)
Test shape: (5000, 2)


In [ ]:
# Split into train/validation (adjust test_size as preferred)
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)

# Get unique labels from training set and create mapping dictionaries.
unique_labels = sorted({label for tags in train_df['NER Tag'] for label in tags})
label2id = {label: i for i, label in enumerate(unique_labels)}
id2label = {i: label for label, i in label2id.items()}

print("Unique labels:", unique_labels)

Unique labels: ['B-art', 'B-eve', 'B-geo', 'B-gpe', 'B-nat', 'B-org', 'B-per', 'B-tim', 'I-art', 'I-eve', 'I-geo', 'I-gpe', 'I-nat', 'I-org', 'I-per', 'I-tim', 'O']


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("answerdotai/ModernBERT-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

In [ ]:
def encode_examples(example):
    # Tokenize input (word-level input, not sentence string)
    tokenized_input = tokenizer(
        example["Sentence"],
        truncation=True,
        padding="max_length",
        max_length=128,
        is_split_into_words=True,
        return_tensors="pt"
    )

    word_ids = tokenized_input.word_ids(batch_index=0)  # for single example
    previous_word_idx = None
    label_ids = []

    for word_idx in word_ids:
        if word_idx is None:
            label_ids.append(-100)
        elif word_idx != previous_word_idx:
            label_ids.append(label2id[example["NER Tag"][word_idx]])
        else:
            label_ids.append(-100)
        previous_word_idx = word_idx

    tokenized_input["labels"] = torch.tensor(label_ids)

    # Remove batch dimension for Hugging Face datasets map compatibility
    return {k: v.squeeze() if isinstance(v, torch.Tensor) else v for k, v in tokenized_input.items()}

In [ ]:
# Convert dataframes to datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Apply tokenization function
train_dataset = train_dataset.map(encode_examples, remove_columns=train_dataset.column_names)
val_dataset = val_dataset.map(encode_examples, remove_columns=val_dataset.column_names)

Map:   0%|          | 0/36000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [ ]:
# Set up the model
model = AutoModelForMaskedLM.from_pretrained(
    "answerdotai/ModernBERT-base",
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id,
    attention_probs_dropout_prob=0.1,
    hidden_dropout_prob=0.1
)

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/599M [00:00<?, ?B/s]

In [ ]:
# Optional: Send model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print("Using device:", device)

Using device: cuda


In [ ]:
# Define metric computation
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_labels = [
        [id2label[label] for label in sent_labels if label != -100]
        for sent_labels in labels
    ]
    true_predictions = [
        [id2label[p] for (p, l) in zip(sent_preds, sent_labels) if l != -100]
        for sent_preds, sent_labels in zip(predictions, labels)
    ]

    # Using seqeval's f1 score
    return {"f1": f1_score(true_labels, true_predictions)}

In [ ]:
# Set training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
    save_total_limit=1,
    push_to_hub=False,
)

# Set up the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

<ipython-input-17-2bf6c179d2e2>:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Training
trainer.train()

Epoch,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 4.42 GiB. GPU 0 has a total capacity of 14.74 GiB of which 4.42 GiB is free. Process 2505 has 10.31 GiB memory in use. Of the allocated memory 9.91 GiB is allocated by PyTorch, and 277.16 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

save_path = '/content/drive/MyDrive/comp4211/'
# os.listdir(save_path)
# Saving both model and tokenizer
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
trainer.save_model("NER_Pro")

Mounted at /content/drive


['PA2_final.ipynb', 'NER_project']

In [ ]:
param_count = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {param_count:,}")

Total parameters: 332,547,089


In [ ]:
# Create a tokenization-only function for test set
def tokenize_test(example):
    tokenized = tokenizer(
        example["Sentence"],
        truncation=True,
        padding="max_length",
        max_length=128,
        is_split_into_words=True,
        return_tensors=None  # leave as lists for HF Dataset
    )
    tokenized["word_ids"] = tokenized.word_ids()  # store the word_ids
    return tokenized

In [ ]:
# 1) Check for duplicate IDs
dupes = test_df["id"][test_df["id"].duplicated()]
if len(dupes):
    print(f"Duplicate IDs found: {dupes.tolist()}")
else:
    print("No duplicate IDs.")

# 2) Check for sentences that get truncated by the tokenizer
too_long = []
for idx, sent in enumerate(test_df["Sentence"]):
    toks = tokenizer(
        sent,
        truncation=True,
        padding=False,
        max_length=128,
        is_split_into_words=True
    )
    # count actual word_ids (excluding special tokens)
    wids = toks.word_ids()
    # words retained = max word_idx + 1
    max_word = max([w for w in wids if w is not None], default=-1) + 1
    if max_word < len(sent):
        too_long.append((test_df["id"].iloc[idx], len(sent), max_word))

if too_long:
    print("Sentences being truncated (id, orig_len, kept_words):")
    for t in too_long[:5]:
        print(" ", t)
else:
    print("No truncation issues (all sentences ≤128 tokens).")


✅ No duplicate IDs.
✅ No truncation issues (all sentences ≤128 tokens).


In [ ]:
# 1) Tokenize test set once, storing word_ids and keeping Sentence
def tokenize_with_word_ids(example):
    # Keep the original sentence for later length check
    sent = example["Sentence"]
    tokenized = tokenizer(
        sent,
        truncation=True,
        padding="max_length",
        max_length=128,
        is_split_into_words=True
    )
    example["word_ids"] = tokenized.word_ids()
    # store tokenized fields
    example.update(tokenized)
    return example

# Build the test dataset, but don't drop 'Sentence' yet
test_dataset = Dataset.from_pandas(test_df)
test_dataset = test_dataset.map(
    tokenize_with_word_ids,
    remove_columns=[]  # keep all columns, including 'Sentence'
)

# 2) Align predictions: but first, prepare inputs only
# Create a view with only the model inputs
pred_dataset = test_dataset.remove_columns(["Sentence", "id"])  # keep only input_ids, attention_mask, token_type_ids, word_ids
pred_dataset.set_format(type="torch")

# 3) Run predictions
raw_preds = trainer.predict(pred_dataset)
preds = np.argmax(raw_preds.predictions, axis=2)

# 4) Align predictions to original words, padding if needed
final_preds = []
for sent, pred_row, word_ids in zip(test_dataset["Sentence"], preds, test_dataset["word_ids"]):
    aligned = []
    prev = None
    for idx, widx in enumerate(word_ids):
        if widx is not None and widx != prev:
            aligned.append(id2label[pred_row[idx]])
        prev = widx

    # If for some reason we have fewer tags than words, pad with 'O'
    if len(aligned) < len(sent):
        padding = ["O"] * (len(sent) - len(aligned))
        aligned.extend(padding)

    # Or if too many (shouldn't happen), truncate
    if len(aligned) > len(sent):
        aligned = aligned[: len(sent)]

    final_preds.append(aligned)

# 5) Sanity-check lengths
mismatches = [
    (ex["id"], len(ex["Sentence"]), len(p))
    for ex, p in zip(test_dataset, final_preds)
    if len(ex["Sentence"]) != len(p)
]
if mismatches:
    print("Still mismatches (should be none):", mismatches)
else:
    print("All lengths match after padding!")

# 6) Build submission
submission_df = pd.DataFrame({
    "id": test_dataset["id"],
    "NER Tag": [str(p) for p in final_preds]
})
submission_df.to_csv("submission_large_bert.csv", index=False)
print("Submission file generated!")
files.download("submission_large_bert.csv")

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

✅ All lengths match after padding!
Submission file generated!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Make submission dataframe; assume test_df has an "id" column
submission_df = pd.DataFrame({
    "id": test_df["id"],
    "NER Tag": final_preds
})

In [ ]:
submission_df["NER Tag"] = submission_df["NER Tag"].apply(str)
submission_df.to_csv("submission_large_bert.csv", index=False)
print("Submission file generated!")

Submission file generated!


In [ ]:
mismatches = []

for idx, (sent, pred) in enumerate(zip(test_df["Sentence"], final_preds)):
    if len(sent) != len(pred):
        mismatches.append((test_df["id"][idx], len(sent), len(pred)))

if mismatches:
    print(f"Found {len(mismatches)} mismatched entries:")
    for mid, slen, plen in mismatches:
        print(f" - ID {mid}: Sentence length = {slen}, Prediction length = {plen}")
else:
    print("All predictions match sentence lengths!")

✅ All predictions match sentence lengths!


In [ ]:
files.download("submission_large_bert.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>